### Planet NDVI time series

In [12]:
# This script is designed to process 4 band multispectral
# satellite images from Planet Labs to calculate the 
# Normalized Difference Water Index (NDVI).

# Classification is being performed as negative NDVI

'''
Perform classification ##### (<= Negative NDVI, else >= NDVI) #####
classified = np.where(ndvi <= ndvi_threshold, 1, nodata).astype(np.uint8)
print(f"Classified Min: {classified.min()}, Max: {classified.max()}")
'''
# This Python script is setup to calculate the surface area of water on the 
# landscape or targeted at individual waterbodies

# Planet imagery across Queensland is covered by 3 UTM zones
# Ensure your AOI projection matches your imagery

# UTM Zone 54S: This zone covers the easternmost part of 
# Queensland, including the eastern coast and cities such as Cairns.

# UTM Zone 55S: Most of central Queensland falls under this zone.

# UTM Zone 56S: The vast majority of southeastern Queensland,
# including Brisbane and the Gold Coast, is within this zone.

# 28/01/2025

# Remote Sensing
# remotesensing@rdmw.qld.gov.au

# Craig Turner
# craig.turner@rdmw.qld.gov.au 
 
# Department of Local Government, Water, and Volunteers
# Water Operations & Systems
# Strategic Compliance, Intelligence, & Investigations

software_version = 0.7

### Import modules

In [13]:
# Add any additional modules to Version information list below

# Standard library imports
import datetime
from datetime import datetime as dt
import os
import sys

# Third-party library imports
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib
matplotlib.use('Agg')  # Prevent inline display of imagery
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg
from PIL import Image, ImageTk
import pytz
from pyproj import CRS
import rasterio
from rasterio.io import MemoryFile
from rasterio.mask import mask
from rasterio.plot import show
import tkinter as tk
from tkinter import filedialog, messagebox, Label, Tk, ttk
import xlsxwriter
from xlsxwriter.workbook import Workbook

# Version information list
modules = {
    'datetime': datetime,
    'os': os,
    'sys': sys,
    'glob': glob,
    'NumPy': np,
    'pandas': pd,
    'GeoPandas': gpd,
    'Maplotlib': plt,    
    'PIL': Image,
    'pytz': pytz,
    'pyproj': CRS,
    'rasterio': rasterio,
    'tkinter': tk,
    'XlsxWriter': xlsxwriter
}

### Declare functions

In [14]:
# Define a function to calculate the covered area in square metres
def calculate_covered_area(udm_band, udm_meta, aoi):
    # Transform the pixel coordinates to the geographic coordinates
    transform = udm_meta['transform']
    
    # Count the number of pixels that are True within the mask (indicating presence of cloud/haze/shadow)
    pixel_area = (transform[0] * -transform[4])  # The area of a pixel in square metres
    covered_pixels = udm_band.sum()
    covered_area_sqm = covered_pixels * pixel_area
    
    return covered_area_sqm

# Function to clip raster to shapefile
def clip_raster_to_shapefile(raster_file, shapefile):
    # Load the boundaries of the clip shapefile using geopandas
    shp = gpd.read_file(shapefile)
    
    # Open the source raster using rasterio
    with rasterio.open(raster_file) as src:
        # Clip the raster with the shapefile using a mask
        out_image, out_transform = mask(src, shp.geometry, crop=True)
        # Copy and update the metadata for the clipped raster
        out_meta = src.meta
        out_meta.update({
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
    
    # Return the in-memory clipped raster data and the updated metadata
    return out_image, out_meta    
    
# Function to classify raster into two classes based on a threshold
def classify_raster(raster_file, threshold, output_file):
    with rasterio.open(raster_file) as src:
        # Read raster band 1
        band1 = src.read(1)

        # Apply classification based on threshold
        classified = np.where(band1 > threshold, 0, 1).astype(np.uint8)

        # Check for existing nodata value in the raster's metadata
        nodata = src.nodata
        if nodata is None:
            nodata = 0  # Default to 0 or another appropriate nodata value for your dataset

        # Apply the classification
        classified = np.where(band1 > threshold, 1, nodata).astype(np.uint8)

        # Update metadata for output
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "dtype": "uint8",
            "nodata": nodata,
            "count": 1,
            "compress": "lzw"
        })

    # Write out the raster
    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(classified, 1)

# Function to calculate area
def measure_areas(masked_raster_band, threshold, pixel_area_m2):
    # Mask for valid data (non-NaN values)
    valid_data_mask = ~np.isnan(masked_raster_band)

    # Masks for areas above and below the threshold
    above_threshold_mask = (masked_raster_band >= threshold) & valid_data_mask
    below_threshold_mask = (masked_raster_band < threshold) & valid_data_mask
    
    # Calculate areas for above and below threshold
    above_threshold_area = np.sum(above_threshold_mask) * pixel_area_m2
    below_threshold_area = np.sum(below_threshold_mask) * pixel_area_m2

    # Calculate the total valid area
    total_valid_area = above_threshold_area + below_threshold_area

    # Calculating the percentages
    percentage_above_threshold = (above_threshold_area / total_valid_area) * 100 if total_valid_area != 0 else 0
    percentage_below_threshold = (below_threshold_area / total_valid_area) * 100 if total_valid_area != 0 else 0

    return total_valid_area, below_threshold_area, percentage_below_threshold, above_threshold_area, percentage_above_threshold

# Function to retrieve projection human readable name
def get_projection_name(epsg_code):
    crs = CRS.from_epsg(epsg_code)
    return crs.name

# Convert UTC time in filename to AEST (UTC+10:00)
def filename_utc_to_aest(filename):
    # Extract the timestamp from the filename, assume the format is: YYYYMMDD_HHMMSS
    file_timestamp_str = filename.split('_')[1]
    # Assuming the date is at the beginning of the filename '20231118'
    file_date_str = filename.split('_')[0]
    file_datetime_str = file_date_str + '_' + file_timestamp_str

    # Create a datetime object
    file_datetime = dt.strptime(file_datetime_str, "%Y%m%d_%H%M%S")

    # Timezone conversion from UTC to AEST
    local_tz = pytz.timezone('Australia/Brisbane')
    local_dt = file_datetime.replace(tzinfo=pytz.utc).astimezone(local_tz)

    # Format the datetime object to a string for full AEST timestamp
    local_dt_full_str = local_dt.strftime('%Y%m%d_%H%M%S')

    # Format the datetime object to a string for just the date portion
    local_dt_date_str = local_dt.strftime('%d/%m/%Y')

    return local_dt_full_str, local_dt_date_str

# GUI stuff, more details to be added
class PlaceholderEntry(tk.Entry):
    def __init__(self, master=None, placeholder="PLACEHOLDER", color='grey', **kwargs):
        super().__init__(master=master, **kwargs)
        self.placeholder = placeholder
        self.placeholder_color = color
        self.default_fg_color = self['fg']

        self.bind("<FocusIn>", self.foc_in)
        self.bind("<FocusOut>", self.foc_out)

        self.put_placeholder()

    def put_placeholder(self):
        self.insert(0, self.placeholder)
        self['fg'] = self.placeholder_color

    def foc_in(self, *args):
        if self.get() == self.placeholder and self['fg'] == self.placeholder_color:
            self.delete('0', 'end')
            self['fg'] = self.default_fg_color

    def foc_out(self, *args):
        if not self.get():
            self.put_placeholder()

def selection_changed(event):
    print("Selected:", combobox.get())

def select_file(entry_widget):
    # Define the file type filter
    filetypes = (
        ('Shapefiles', '*.shp'),
    )

    # Open file dialog with .shp filter
    file_path = filedialog.askopenfilename(title="Select file", filetypes=filetypes)

    # Check if a file was selected
    if file_path:
        entry_widget.delete(0, tk.END)  # Remove any existing text in the entry
        entry_widget.insert(0, file_path)  # Insert the selected path

def select_directory(entry_widget):
    directory_path = filedialog.askdirectory(title="Select directory")
    entry_widget.delete(0, tk.END)
    entry_widget.insert(0, directory_path)

def submit_form():
    global job_id, pfi, analyst, ndvi_threshold, colour_map, base_dir, aoi, gui_path

    job_id = field1_entry.get()
    pfi = field2_entry.get()
    analyst = name_combobox.get()
    colour_map = colourmap_combobox.get()
    base_dir = source1_entry.get()
    aoi = source2_entry.get()
    gui_path = output1_entry.get()

    # Get the NDVI threshold as string and then convert it to a float after stripping whitespace.
    ndvi_threshold_str = field4_entry.get().strip()

    try:
        # Convert the threshold value to float and check its range.
        ndvi_threshold = float(ndvi_threshold_str)
        if not -1.0 <= ndvi_threshold <= 1.0:
            messagebox.showerror("Input Error", "Invalid NDVI Threshold value. Please enter a value between -1 and 1.")
            return
    except ValueError:
        messagebox.showerror("Input Error", "Invalid NDVI Threshold value. Please enter a numerical value.")
        return
    
    # After submitting the form and processing the data without any errors, close the GUI.
    root.destroy()

### GUI - Create output folder - Set variables

In [ ]:
# Clear input variables
job_id = None
pfi = None
analyst = None
ndvi_threshold = None
colour_map = None 
base_dir = None
aoi = None
gui_path = None

# Option lists
colour_map_options = ['viridis', 'plasma', 'inferno', 'magma', 'cividis', 
                      'viridis_r', 'plasma_r', 'inferno_r', 'magma_r', 'cividis_r']
name_options = ['Jason Dechastel', 'Lasinidu Jayarathna', 'Eva Kovaks', 'Craig Turner']

# Create corporate colours
Cybernetic = "#275c6d"  # Primary
Prosperity = "#077297"  # Secondary
Bold_Blue = "#023a57"   # Secondary

Mangrove = "#4b623b"    # Primary
Waterhole = "#a69b5e"   # Secondary
H20 = "#315450"         # Secondary

Terrain = "#8c4d36"     # Primary
Ochre = "#ac6d29"       # Secondary
Sunrise = "#c48c33"     # Secondary

# Assign colours
main_bg_colour =  Mangrove 
frame_bg_colour = Waterhole
label_bg_colour = Waterhole
entry_bg_colour = "black" # Black entry widget background
button_bg_colour = "white"  # White button

# Main window creation and setup
root = tk.Tk()
root.title("Planet NDVI time series")
root.configure(bg=main_bg_colour)
root.attributes('-topmost', True)
               
# Load an image using PIL's Image
image_path = 'Qld-CoA-Stylised-1L-mono_rev.png'
image = Image.open(image_path)
photo = ImageTk.PhotoImage(image)

# Create a frame to hold the image and the title
header_frame = tk.Frame(root, bg = frame_bg_colour)
header_frame.grid(row=0, column=0, columnspan=3, sticky="nw", padx=5, pady=5)

# Display Queensland Coat of Arms in the top-left corner of the GUI
image_label = tk.Label(header_frame, image=photo, bg=frame_bg_colour)
image_label.grid(row=0, column=0, sticky="nw")

# Display title to the right of the Queensland Coat of Arms
title_label = tk.Label(header_frame, text="Planet NDVI time series\n \nStrategic Compliance, Intelligence, & Investigations ", font=("Arial", 22), fg="#ffffff", bg=label_bg_colour)
title_label.grid(row=0, column=1, sticky="w")

# Create labels and entry widgets for each field with placeholders
tk.Label(root, text="Job ID:").grid(row=1, column=0, sticky="e")
field1_entry = PlaceholderEntry(root, placeholder="e.g., 231132", width=28)
field1_entry.grid(row=1, column=1, padx=5, pady=5, sticky="w")

tk.Label(root, text="PFI:").grid(row=2, column=0, sticky="e")
field2_entry = PlaceholderEntry(root, placeholder="Persistent Feature Identifier", width=28)
field2_entry.grid(row=2, column=1, padx=5, pady=5, sticky="w")

ttk.Label(root, text="Name:").grid(row=3, column=0, sticky="e")
name_combobox = ttk.Combobox(root, values=name_options, width=25)
name_combobox.grid(row=3, column=1, padx=5, pady=5, sticky="w")
name_combobox.set('Craig Turner')  # Set a default value

tk.Label(root, text="NDVI Threshold:").grid(row=4, column=0, sticky="e")
field4_entry = PlaceholderEntry(root, placeholder="0", width=28)
field4_entry.grid(row=4, column=1, padx=5, pady=5, sticky="w")

ttk.Label(root, text="Matplotlib colourmap:").grid(row=5, column=0, sticky="e")
colourmap_combobox = ttk.Combobox(root, values=colour_map_options, width=25)
colourmap_combobox.grid(row=5, column=1, padx=5, pady=5, sticky="w")
colourmap_combobox.set('viridis')  # Set a default value

tk.Label(root, text="Imagery:").grid(row=6, column=0, sticky="e")
source1_entry = PlaceholderEntry(root, placeholder="C:/Local Imagery/241809-241810-241689/TOA", width=40)
source1_entry.grid(row=6, column=1, padx=5, pady=5, sticky="we")
tk.Button(root, text="Browse", command=lambda: select_directory(source1_entry)).grid(row=6, column=2, padx=5, pady=5, sticky="w")

tk.Label(root, text="Shapefile:").grid(row=7, column=0, sticky="e")
source2_entry = PlaceholderEntry(root, placeholder="C:/Local Shapefiles/241779 Ballangarry AOI/SCP_ExportFeatures.shp", width=40)
source2_entry.grid(row=7, column=1, padx=5, pady=5, sticky="we")
tk.Button(root, text="Browse", command=lambda: select_file(source2_entry)).grid(row=7, column=2, padx=5, pady=5, sticky="w")

tk.Label(root, text="Output:").grid(row=8, column=0, sticky="e")
output1_entry = PlaceholderEntry(root, placeholder="c:\Local Output", width=40)
output1_entry.grid(row=8, column=1, padx=5, pady=5, sticky="we")
tk.Button(root, text="Browse", command=lambda: select_directory(output1_entry)).grid(row=8, column=2, padx=5, pady=5, sticky="w")

# Submit button creation
submit_button = tk.Button(root, text="Submit", command=submit_form)
submit_button.grid(row=9, column=0, columnspan=3, pady=10)

# Display Remote Sensing email address in GUI
title_label = tk.Label(root, text="remotesensing@rdmw.qld.gov.au", font=("Arial", 10), fg="#ffffff", bg=main_bg_colour)
title_label.grid(row=10, column=0, sticky="w")

# Display software version in GUI
title_label = tk.Label(root, text=f"Version {software_version}", font=("Arial", 10), fg="#ffffff", bg=main_bg_colour)
title_label.grid(row=10, column=2, sticky="se")

# Start the GUI event loop
root.mainloop()

# Only read analytical imagery
image_files = glob.glob(os.path.join(base_dir, "*Analytic*.tif"))

# Get current date and time
current_datetime = datetime.datetime.now()

# Format date and time into a string
formatted_datetime = current_datetime.strftime('%Y%m%d_%H%M%S')

# Define the name of the output subfolder
subfolder_name = f"{formatted_datetime}_Job_ID_{job_id}_Threshold_{ndvi_threshold}_Planet NDVI time series"

# Create the full path for the new subfolder
output_path = os.path.join(gui_path, subfolder_name)

# Create the subfolder
os.makedirs(output_path)

# Output Excel
excel_filename = f"{formatted_datetime}_Job_ID_{job_id}_Threshold_{ndvi_threshold}_Planet_NDVI_time_series.xlsx"
excel_file_path = os.path.join(output_path, excel_filename)

# Specify the buffer distance (in units of the raster's CRS)
buffer_distance = 50  # Adjust this value as needed

# Collect Python version information
python_version = sys.version
python_environment = sys.executable

### Check projection

In [ ]:
# Open shapefile using geopandas
shp = gpd.read_file(aoi)

# Read the first tiff file:    
raster_image = rasterio.open(image_files[1])
tiff_band_1 = raster_image.read(1)

# Print shape and raster CRS info:
# print ('Check to ensure shapefile and raster projections match\n')

# Check and get the projection name if shp.crs has an EPSG code
shp_epsg_code = shp.crs.to_epsg()
if shp_epsg_code:
    shp_projection_name = get_projection_name(shp_epsg_code)
    print(f"The projection name for the shapefile with EPSG code {shp_epsg_code} is, {shp_projection_name}.\n")
else:
    print('The shapefile CRS does not have an EPSG code.')

# Check and get the projection name if raster_image.crs has an EPSG code
raster_image_epsg_code = raster_image.crs.to_epsg()
if raster_image_epsg_code:
    raster_image_projection_name = get_projection_name(raster_image_epsg_code)
    print(f"The projection name for the raster image with EPSG code {raster_image_epsg_code} is, {raster_image_projection_name}.\n")
else:
    print('The raster image CRS does not have an EPSG code.')

# Check if projections match
if shp.crs != raster_image.crs:
    print('Projections do not match.')
else:
    print('Projections match.')
    
# Get raster extent
raster_extent = [raster_image.bounds[0], raster_image.bounds[2], raster_image.bounds[1], raster_image.bounds[3]]

# Create a new plot
f, ax = plt.subplots()

# Render image
show(
    tiff_band_1,
    extent=raster_extent,
    ax=ax,
)

# Add shapefile
shp.plot(ax=ax, facecolor='w', edgecolor='k')
ax.axis('off')
ax.set_title('Projection Check')

# Save image to output folder
figure_filename = f"{formatted_datetime}_Job_ID_{job_id}_PFI_{pfi}_Planet_NDVI_time_series.jpg"
figure_save_file_path = os.path.join(output_path, figure_filename)
plt.savefig (figure_save_file_path)

### Loop through images

In [ ]:
# Initialise empty DataFrame
df = pd.DataFrame()

# Record start
start_time = datetime.datetime.now()
print("Starting at " + start_time.strftime("%d/%m/%Y, %H:%M:%S"))

# Time variables
start_time_str = start_time.strftime("%d/%m/%Y, %H:%M:%S")

# Loop through image folder
for image_file in image_files:
    # Get base image name
    image_name = os.path.basename(image_file)
    # UDM file name
    udm_file = image_file.replace('AnalyticMS_SR', 'udm2').replace('AnalyticMS', 'udm2')
    
    # Convert the image file timestamp from UTC to AEST
    aest_file_timestamp , aest_date = filename_utc_to_aest(image_name)    

    # Prepend the converted timestamp to the original filename
    output_image_name = f"{aest_file_timestamp}_AEST_{image_name}"
    
    # Get image date
    image_date = image_name[:8]
    image_date = dt.strptime(image_date, '%Y%m%d').strftime('%d/%m/%Y')

    # Clip image and get the clipped image data and metadata in memory
    clipped_data, clipped_meta = clip_raster_to_shapefile(image_file, aoi)

    # Assume nodata as None before getting the actual value from metadata
    nodata = None
    
    # Clip AOI to buffer
    shp = gpd.read_file(aoi)
    
    # Open the image with rasterio
    with rasterio.open(image_file) as src:
        # Buffer the AOI and then clip
        buffered_aoi = shp.geometry.buffer(buffer_distance)
        out_image, out_transform = rasterio.mask.mask(src, buffered_aoi, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        # Define the output filename using your naming convention
        buffered_output = os.path.join(output_path, f"{output_image_name}_RGB_Buffered.tif")
        
        # Save the buffered RGB clip to a new GeoTIFF file
        with rasterio.open(buffered_output, "w", **out_meta) as dest:
            dest.write(out_image)    

    print("\nProcessing date: " + image_date)

    # print("Image name: " + image_name)
    # print('UDM file: ', udm_file)
    with MemoryFile() as memfile:
        with memfile.open(**clipped_meta) as dataset:
            # Retrieve the nodata value from the dataset
            nodata = dataset.nodata
            
            # print(f"Clipped Data Shape: {dataset.shape}")  # (height, width)
            # print(f"Clipped Data Transform: {dataset.transform}")

            # Write the raster data to the dataset
            dataset.write(clipped_data)

            # Now, perform operations using the dataset as a proxy to the data
            # abs to return only positive values that make sense to user
            transform = dataset.transform
            x_res = abs(transform.a) # Resolution in x direction (transform[0])
            y_res = abs(transform.e)  # Resolution in y direction (transform[4])
            pixel_area_m2 = (x_res * y_res) 
            
            # Load red and NIR bands for NDVI calculation
            bandred = dataset.read(3).astype(float)
            bandnir = dataset.read(4).astype(float)    

            # Allow division by zero
            np.seterr(divide='ignore', invalid='ignore')
            
            # Ensures 'ndvi_threshold' is a number.
            if ndvi_threshold is None:
                raise ValueError("NDVI threshold is 'None', expected a float.")

            # Ensures 'bandred' and 'bandnir' contain no 'None' values.
            if np.any(bandred == None) or np.any(bandnir == None):
                raise ValueError("Band arrays contain 'None' values, expected only numerical data.")            

            # Calculate NDVI
            ndvi = (bandnir - bandred) / (bandnir + bandred)
            
            total_valid_area, below_threshold_area, percentage_below_threshold, above_threshold_area, percentage_above_threshold = measure_areas(ndvi, ndvi_threshold, pixel_area_m2)

            # Define nodata value
            nodata = 0
            # nodata = dataset.nodata if dataset.nodata is not None else 0

            # Perform classification ##### (<= Negative NDVI, else >= NDVI) #####
            classified = np.where(ndvi <= ndvi_threshold, 1, nodata).astype(np.uint8)
            # print(f"Classified Min: {classified.min()}, Max: {classified.max()}")

            # Create a colormap (0: transparent, 1: blue)
            colormap = {0: (0, 0, 0, 0), 1: (0, 0, 255, 255)}
            
            # Define metadata for the new single-band GeoTIFF with classification
            out_meta = dataset.meta.copy()
            out_meta.update({
                'driver': 'GTiff',
                'dtype': 'uint8',
                'count': 1,
                'compress': 'lzw',
                'nodata': nodata,
                'photometric': 'RGBA'
            })

        # Print and check dimensions and transform before writing
        # print(f"Writing Classified Raster with Size: {classified.shape}")  # Should match dataset.shape
        # print(f"Output Data Transform: {out_meta['transform']}")

        # Before writing, check dimensions and potential issues
        if classified.shape != (dataset.height, dataset.width):
            print("Error: Classified raster dimensions do not match dataset dimensions.")
        elif classified.max() == nodata:
            print("Error: All classified raster values are nodata.")
        
        # Define the output path
        classified_output = os.path.join(output_path, f"{output_image_name}_Threshold_{ndvi_threshold}_NDVI_Classified.tif")  # Use os.path.join for cross-platform compatibility

        # Set spatial characteristics of the output object to mirror the input
        kwargs = dataset.meta
        kwargs.update(
        dtype=rasterio.float32,
        count = 1
        )

        # Write out the classified raster with colormap applied
        with rasterio.open(classified_output, 'w', **out_meta) as dst:
            dst.write(classified, 1)
            dst.write_colormap(1, colormap)
        
        
            # Read UDM file for cloud, haze, and shadow
            # Define metadata for a new single-band GeoTIFF
            ndvi_meta = dataset.meta.copy()
            ndvi_meta.update({
                'dtype': 'uint8',
                'count': 1,
                'compress': 'lzw',
                'nodata': 0  # Set the nodata value as required
            })
            
            # Clip the UDM file
            with rasterio.open(udm_file) as src:
                cloud_mask = src.read(6).astype(bool) # cloud is 6th band (index 5)
                haze_mask = src.read(4).astype(bool) # haze is 4th band (index 3)
                shadow_mask = src.read(3).astype(bool) # shadow is 3rd band (index 2)
                extent = rasterio.plot.plotting_extent(src)

            # Use rasterio's geometry mask to clip the UDM to the area of interest
            with rasterio.open(udm_file) as src:
                out_image, out_transform = rasterio.mask.mask(src, shp.geometry, crop=True)
                out_meta = src.meta.copy()
                out_meta.update({'driver': 'GTiff',
                                'height': out_image.shape[1],
                                'width': out_image.shape[2],
                                'transform': out_transform})
                
                # Calculate the covered area for each mask
                cloud_covered_area = calculate_covered_area(out_image[5], out_meta, shp)
                haze_covered_area = calculate_covered_area(out_image[3], out_meta, shp)
                shadow_covered_area = calculate_covered_area(out_image[2], out_meta, shp)

                # Calculate cover percentages
                cloud_covered_percentage = (cloud_covered_area / total_valid_area) * 100
                haze_covered_percentage = (haze_covered_area / total_valid_area) * 100
                shadow_covered_percentage = (shadow_covered_area / total_valid_area) * 100

                # Print the results
                # print(f"Total area (sqm): {total_valid_area}")
                # print(f"Cloud covered area (sqm): {cloud_covered_area}")
                # print(f"Haze covered area (sqm): {haze_covered_area}")
                # print(f"Shadow covered area (sqm): {shadow_covered_area}")

                # Print cover percentages
                print(f"Cloud covered %: {cloud_covered_percentage}")
                print(f"Haze covered %: {haze_covered_percentage}")
                print(f"Shadow covered %: {shadow_covered_percentage}")

    # Define kwargs here
    kwargs = dataset.meta.copy()
    kwargs.update(
        dtype=rasterio.float32,
        count=1)
    
    # Write out NDVI GeoTIFF image
    ndvi_output = output_path + "\\" + output_image_name + "_NDVI.tif"
    with rasterio.open(ndvi_output, 'w', **kwargs) as dst:
        dst.write_band(1, ndvi.astype(rasterio.float32))   

    # Prepare the appropriate metadata
    out_meta = dataset.meta.copy()
    out_meta.update({
        'driver': 'GTiff',
        'height': classified.shape[0],  # Update to match classification array's height
        'width': classified.shape[1],   # Update to match classification array's width
        'transform': dataset.transform, # Update if you've altered the transform
        'dtype': 'uint8',
        'count': 1,  # Single-band raster
        'compress': 'lzw',
        'nodata': nodata  # Nodata value
    })
    
    # print(image_date)
    
    # Use NumPy to mask out invalid pixels
    ndvi_masked = np.ma.masked_invalid(ndvi)
    
    # Define the dimensions of the image
    ndvi_shape = ndvi_masked.shape

    # Create the plot area
    fig, ax = plt.subplots(figsize=(12, 12))

    # Create a new figure with the same dimensions as ndvi_masked
    fig = Figure(figsize=(ndvi_shape[1]/fig.dpi, ndvi_shape[0]/fig.dpi), dpi=fig.dpi)
    canvas = FigureCanvasAgg(fig)
    ax = fig.add_axes([0, 0, 1, 1])  # Add an axis that covers the entire figure

    # Show the masked NDVI image without padding and without axes
    ax.imshow(ndvi_masked, cmap=colour_map)
    ax.axis('off')

    # Add your image_date text to the image at the top-left corner
    timestamp_with_timezone = f"{aest_file_timestamp} AEST"  # Append 'AEST' to the timestamp
    text_props = dict(boxstyle='round', facecolor='black', alpha=0.5)
    ax.text(0.01, 0.99, timestamp_with_timezone, transform=ax.transAxes, fontsize=12, color='white', bbox=text_props, verticalalignment='top', horizontalalignment='left')

    # Save the figure to a file with the given path and name
    output_file_path = f"{output_path}\\{output_image_name}_NDVI_Colour.png"
    fig.savefig(output_file_path, dpi=fig.dpi, bbox_inches='tight', pad_inches=0, transparent=True)

    # We no longer need the figure after saving it, free the memory
    plt.close(fig)
    
    # Gather statistics
    bandred_min = np.nanmin(bandred)
    bandred_max = np.nanmax(bandred)
    bandred_mean = np.nanmean(bandred)
           
    bandnir_min = np.nanmin(bandnir)
    bandnir_max = np.nanmax(bandnir)
    bandnir_mean = np.nanmean(bandnir)
               
    ndvi_min = np.nanmin(ndvi)
    ndvi_max = np.nanmax(ndvi)
    ndvi_mean = np.nanmean(ndvi)
        
    # Append results to DataFrame
    new_row = pd.DataFrame({
        "Image Name": [output_image_name + "_NDVI.tif"],
        "Image Date AEST": [aest_date],
        "Total Area (sqm)": [total_valid_area],
        "Total Area (ha)": [total_valid_area/10000],
        "NDVI Area (sqm)": [above_threshold_area],
        "NDVI Area (ha)": [above_threshold_area/10000],
        "NDVI %": [percentage_above_threshold],
        "-NDVI Area (ha)": [(total_valid_area-above_threshold_area)/10000],
        "-NDVI %": [100-percentage_above_threshold],
        "Cloud Coverage": [cloud_covered_area],
        "Cloud %": [cloud_covered_percentage],
        "Haze (sqm)": [haze_covered_area],
        "Haze %": [haze_covered_percentage],
        "Shadow (sqm)": [shadow_covered_area],
        "Shadow %": [shadow_covered_percentage],
        "NDVI Min": [ndvi_min],
        "NDVI Max": [ndvi_max],
        "NDVI Mean": [ndvi_mean],
        "Red Band Min": [bandred_min],
        "Red Band Max": [bandred_max],
        "Red Band Mean": [bandred_mean],
        "NIR Band Min": [bandnir_min],
        "NIR Band Max": [bandnir_max],
        "NIR Band Mean": [bandnir_mean],
    })
    df = pd.concat([df, new_row], ignore_index=False)

# Record end
end_time = datetime.datetime.now()
end_time_str = end_time.strftime("%d/%m/%Y, %H:%M:%S")

# Calculate processing time
processing_time = end_time - start_time
processing_time_str = str(processing_time)

print("\nEntire period processed, started at " + start_time_str + " and finished at " + end_time_str + ".\n")
print("Total processing time: " + processing_time_str + ".")

### Write metadata outputs

In [ ]:
# Create a DataFrame with process metadata
metadata_df = pd.DataFrame({
    "Variable Name": ["Job ID",
                      "PFI",
                      "Analyst",
                      "NDVI Threshold",
                      "Matplotlib Colourmap",
                      "Start Date and Time",
                      "End Date and Time",
                      "Total Time HH:MM:SS.ss",
                      "Shapefile EPSG Code",
                      "Shapefile Projection Name",
                      "Raster EPSG Code",
                      "Raster Projection Name",
                      "X Pixel Resolution (m)",
                      "Y Pixel Resolution (m)",
                      "Pixel Area (sqm)",
                      "Source Imagery",
                      "Source Shapefile",
                      "Output Folder",
                      "Software Version",
                      "Python Version",
                      "Python Environment"],
    "Value": [job_id,
                pfi,
                analyst,
                ndvi_threshold,
                colour_map,
                start_time_str,
                end_time_str,
                processing_time_str,
                shp_epsg_code,
                shp_projection_name,
                raster_image_epsg_code,
                raster_image_projection_name,
                x_res,
                y_res,
                pixel_area_m2,
                base_dir,
                aoi,
                output_path,
                software_version,
                python_version,
                python_environment]
})

# Create a dictionary to store the module names and versions
module_versions = {}

# Iterate over the modules dictionary in and get the module name and version
for module_name, module in modules.items():
    try:
        if module_name == 'Python':
            version = sys.version.split()[0]
        else:
            version = module.__version__
    except AttributeError:
        version = 'Same as Python version'
    module_versions[module_name] = version

# Create a pandas DataFrame from the dictionary
modules_df = pd.DataFrame(list(module_versions.items()), columns=['Module', 'Version'])

# ExcelWriter to create an Excel output, add NDVI Output, Job Metadata, and Python Modules tabs
writer = pd.ExcelWriter(excel_file_path, engine="xlsxwriter")
workbook = writer.book

# Add Job NDVI Output tab
df.to_excel(writer, sheet_name="NDVI Output", index=False)
# NDVI Output worksheet
ndvi_output_worksheet = writer.sheets['NDVI Output']
ndvi_output_worksheet.autofit()

# Add Job Metadata tab
metadata_df.to_excel(writer, sheet_name='Job Metadata', index=False, header=False)
# Job Metadata worksheet
metadata_worksheet = writer.sheets['Job Metadata']
left_align_format = workbook.add_format({'align': 'left'})
metadata_worksheet.set_column('B:B', None, left_align_format)
metadata_worksheet.autofit() 
# Add Python module versions
modules_df.to_excel(writer, sheet_name='Python Modules', index=False, header=False)
# Python Modules worksheet
python_modules_worksheet = writer.sheets['Python Modules']
python_modules_worksheet.autofit()

# Close the Pandas ExcelWriter and save the Excel file
writer.close()

print('Excel spreadsheet containing job information is here:\n')
print('Excel file path:', excel_file_path)